# Visualización de Información Demográfica y Pruebas PR/PCR

### 1. Tratamiento y Visualización de Datos

#### 1.1 Importamos librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import json
import plotly.graph_objects as go
import plotly.express as px


from dataclasses import dataclass
from typing import *

#### 1.2 Cargamos los datos

In [2]:
positivos = pd.read_csv('dataset/positivos_covid.csv')
fallecidos = pd.read_csv('dataset/fallecidos_covid.csv')
departamentos = pd.read_csv('dataset/departamentos_peru_v2.csv')

In [3]:
# Agregamos columna de longitud y latitud
positivos = pd.merge(positivos, departamentos, how='left', left_on='DEPARTAMENTO', right_on='DEPARTAMENTO')
# Agregamo columna para verificar el rango de edad
"""
    Según MINSA: 
        Niño            [0-11]
        Adolescente     [12-17]
        Joven           [18-29]
        Adulto          [30-59]
        Adulto Mayor    [60-más]
    
    Fuente: http://www.minsa.gob.pe/estadisticas/estadisticas/indicadoresSalud/demograficos/poblaciontotal/POBVIDMacros.asp
    
"""

def create_group_age(age):
    if np.isnan(age):
        return np.nan
    elif age >= 0. and age <=11.:
        return 'NIÑO'
    elif age >= 12. and age <=17.:
        return 'ADOLESCENTE'
    elif age >= 18. and age <=29.:
        return 'JOVEN'
    elif age >= 30. and age <=59.:
        return 'ADULTO'
    else:
        return 'ADULTO MAYOR'

def create_range_group_age(age):
    if np.isnan(age):
        return np.nan
    elif age >= 0. and age <=11.:
        return '0 a 11 años'
    elif age >= 12. and age <=17.:
        return '12 a 17 años'
    elif age >= 18. and age <=29.:
        return '18 a 29 años'
    elif age >= 30. and age <=59.:
        return '30 a 59 años'
    else:
        return '60 años a más'    

positivos['GRUPO_EDAD'] = positivos['EDAD'].apply(create_group_age)
positivos['RANGO_EDAD'] = positivos['EDAD'].apply(create_range_group_age)

#### Dataset Positivos

In [4]:
positivos.drop('UUID', axis=1, inplace=True)
positivos[pd.notnull(positivos['FECHA_RESULTADO'])].sort_values(by='FECHA_RESULTADO')

DEPARTAMENTO         PROVINCIA           DISTRITO METODODX  EDAD  \
184465       CALLAO            CALLAO         BELLAVISTA      PCR  68.0   
70789        CALLAO            CALLAO             CALLAO       PR  35.0   
165420       CALLAO            CALLAO             CALLAO      PCR  59.0   
159370        JUNIN          HUANCAYO           EL TAMBO      PCR  57.0   
159371       LORETO            MAYNAS           PUNCHANA      PCR  57.0   
...             ...               ...                ...      ...   ...   
31001          LIMA              LIMA                ATE       PR  26.0   
148405          ICA           CHINCHA       CHINCHA ALTA       PR  53.0   
128088  LA LIBERTAD            CHEPEN             CHEPEN       PR  48.0   
193969         LIMA              LIMA  MAGDALENA DEL MAR       PR  75.0   
13397       UCAYALI  CORONEL PORTILLO        YARINACOCHA       PR  20.0   

             SEXO FECHA_RESULTADO   LATITUDE  LONGITUDE    GRUPO_EDAD  \
184465  MASCULINO       1/04/2020 -12.063433 -77.147186  ADULTO MAYOR   
70789   MASCULINO       1/04/2020 -12.063433 -77.147186        ADULTO   
165420   FEMENINO       1/04/2020 -12.063433 -77.147186        ADULTO   
159370  MASCULINO       1/04/2020 -11.161393 -75.997976        ADULTO   
159371  MASCULINO       1/04/2020  -3.749235 -73.244396        ADULTO   
...           ...             ...        ...        ...           ...   
31001   MASCULINO       9/05/2020 -12.045662 -77.030570         JOVEN   
148405  MASCULINO       9/05/2020 -14.063867 -75.729108        ADULTO   
128088  MASCULINO       9/05/2020  -8.111596 -79.028716        ADULTO   
193969   FEMENINO       9/05/2020 -12.045662 -77.030570  ADULTO MAYOR   
13397    FEMENINO       9/05/2020  -9.037084 -73.508659         JOVEN   

           RANGO_EDAD  
184465  60 años a más  
70789    30 a 59 años  
165420   30 a 59 años  
159370   30 a 59 años  
159371   30 a 59 años  
...               ...  
31001    18 a 29 años  
148405   30 a 59 años  
128088   30 a 59 años  
193969  60 años a más  
13397    18 a 29 años  

[202277 rows x 11 columns]

#### Dataset Fallecidos

In [5]:
fallecidos_departamento = fallecidos.groupby(by=['DEPARTAMENTO']).count()
fallecidos_departamento.rename(columns={'UUID':'TOTAL_MUERTOS'}, inplace=True)
fallecidos_departamento.reset_index(inplace=True)

#### Creación de sumarizado general

In [6]:
# Total de Casos detectados
total_casos = positivos.groupby(by=['DEPARTAMENTO']).count()
total_casos = total_casos.rename(columns={'PROVINCIA': 'TOTAL_CASOS'})
total_casos.reset_index(inplace=True)

# Total de pruebas rápidas
total_pruebas_rapidas = positivos[positivos['METODODX'] == 'PR'].groupby(by=['DEPARTAMENTO', 'METODODX']).count()
total_pruebas_rapidas = total_pruebas_rapidas.rename(columns={'PROVINCIA': 'PR'})
total_pruebas_rapidas.reset_index(inplace=True)

# Total de pruebas moleculares
total_pruebas_pcr = positivos[positivos['METODODX'] == 'PCR'].groupby(by=['DEPARTAMENTO', 'METODODX']).count()
total_pruebas_pcr = total_pruebas_pcr.rename(columns={'PROVINCIA': 'PCR'})
total_pruebas_pcr.reset_index(inplace=True)

resume = total_casos[['DEPARTAMENTO', 'TOTAL_CASOS']]
resume = pd.merge(resume, 
                  fallecidos_departamento[['DEPARTAMENTO','TOTAL_MUERTOS']],
                  how='left', 
                  left_on='DEPARTAMENTO',
                  right_on='DEPARTAMENTO')

resume = pd.merge(resume, 
                  total_pruebas_rapidas[['DEPARTAMENTO','PR']],
                  how='left', 
                  left_on='DEPARTAMENTO',
                  right_on='DEPARTAMENTO')

resume = pd.merge(resume, 
                  total_pruebas_pcr[['DEPARTAMENTO','PCR']],
                  how='left', 
                  left_on='DEPARTAMENTO',
                  right_on='DEPARTAMENTO')

resume['LETALIDAD'] = resume['TOTAL_MUERTOS'] * 100 / resume['TOTAL_CASOS']

resume.sort_values(by='PR', ascending=False, inplace=True)

#### Pruebas Rápidas por Departamento en Mapa

In [7]:
resume['COD'] = resume['DEPARTAMENTO'].apply(lambda x: ''.join(x.split(' '))[:5])
resume = pd.merge(resume, departamentos, how='left', left_on='DEPARTAMENTO', right_on='DEPARTAMENTO')
resume.rename(columns={'DEPARTAMENTO': 'NOMBDEP'}, inplace=True)

In [8]:
with open('geojson/peru_departamental_simple.geojson') as response:
    peru_geo = json.load(response)

    fig = px.choropleth(resume, geojson=peru_geo, color="PCR", color_continuous_scale="Reds",
                    locations="NOMBDEP", featureidkey="properties.NOMBDEP")
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title_text = 'Pruebas Rápidas por Departamento')
fig.show()

#### Contagios y fallecidos por Rango de Edades y Sexo

In [10]:
fallecidos.drop(['UUID'], axis=1, inplace=True)
fallecidos['GRUPO_EDAD'] = fallecidos['EDAD_DECLARADA'].apply(create_group_age)
fallecidos['RANGO_EDAD'] = fallecidos['EDAD_DECLARADA'].apply(create_range_group_age)

In [11]:
fallecidos.sample(5)

FECHA_FALLECIMIENTO  EDAD_DECLARADA       SEXO   FECHA_NAC DEPARTAMENTO  \
2034           4/05/2020              57   FEMENINO   6/09/1962       ANCASH   
1656          27/04/2020              82  MASCULINO   5/02/1938       CALLAO   
2855          28/04/2020              67  MASCULINO  24/03/1953       LORETO   
4916           3/06/2020              73  MASCULINO  29/12/1946     AREQUIPA   
1286          12/04/2020              62   FEMENINO  30/03/1958         LIMA   

     PROVINCIA        DISTRITO    GRUPO_EDAD     RANGO_EDAD  
2034     SANTA        CHIMBOTE        ADULTO   30 a 59 años  
1656    CALLAO          CALLAO  ADULTO MAYOR  60 años a más  
2855       NaN           BELEN  ADULTO MAYOR  60 años a más  
4916  AREQUIPA  CERRO COLORADO  ADULTO MAYOR  60 años a más  
1286      LIMA     EL AGUSTINO  ADULTO MAYOR  60 años a más

In [12]:
#Contagios por Ranao de Edades y departamento
group = positivos.groupby(by=['DEPARTAMENTO', 'GRUPO_EDAD', 'RANGO_EDAD', 'SEXO']).count()
group.rename(columns={'PROVINCIA': 'TOTAL_POSITIVOS'}, inplace=True)
positivos_dep_age = group[['TOTAL_POSITIVOS']].reset_index()

# Fallecidos por Rango de Edades y departamento
group = fallecidos.groupby(by=['DEPARTAMENTO', 'GRUPO_EDAD', 'RANGO_EDAD', 'SEXO']).count()
group.rename(columns={'FECHA_FALLECIMIENTO': 'TOTAL_FALLECIDOS'}, inplace=True)
fallecidos_dep_age = group[['TOTAL_FALLECIDOS']].reset_index()

# Conjut de datos demográficos
demographic = pd.merge(positivos_dep_age, 
                       fallecidos_dep_age, 
                       how='left', 
                       left_on=['DEPARTAMENTO', 'GRUPO_EDAD', 'RANGO_EDAD', 'SEXO'], 
                       right_on=['DEPARTAMENTO', 'GRUPO_EDAD', 'RANGO_EDAD', 'SEXO'])

demographic['TOTAL_FALLECIDOS'].fillna(0, inplace=True)
demographic['TOTAL_FALLECIDOS'] = demographic['TOTAL_FALLECIDOS'].astype(int)

In [13]:
demographic['COD'] = demographic['DEPARTAMENTO'].apply(lambda x: ''.join(x.split(' '))[:5])

In [14]:
gg = demographic.groupby(by=['COD', 'DEPARTAMENTO', 'GRUPO_EDAD']).sum()
gg = gg.reset_index()
gg.head(20)

names = gg['COD'].unique()
depa = gg['DEPARTAMENTO'].unique()
ages = gg['GRUPO_EDAD'].unique()
values = []
for age in ages:
    vals = []
    for name in names:
        vals.append(gg[((gg['COD'] == name)&(gg['GRUPO_EDAD'] == age))]['TOTAL_FALLECIDOS'].values[0])
    values.append(vals)

bars = []    
for age, value in zip(ages, values):
    bars.append(go.Bar(name=age, x=depa, y=value))

fig = go.Figure(data=bars)
# Change the bar mode
fig.update_layout(title= 'Gráfico de Fallecidos por Grupo de Edad', barmode='stack', xaxis_tickangle=-45)
fig.show()    

In [15]:
cr = demographic.groupby(by=['DEPARTAMENTO','GRUPO_EDAD']).sum()
cr.reset_index(inplace=True)
cr.head(13)

DEPARTAMENTO    GRUPO_EDAD  TOTAL_POSITIVOS  TOTAL_FALLECIDOS
0      AMAZONAS   ADOLESCENTE               30                 0
1      AMAZONAS        ADULTO              511                11
2      AMAZONAS  ADULTO MAYOR               86                15
3      AMAZONAS         JOVEN              204                 1
4      AMAZONAS          NIÑO               39                 0
5        ANCASH   ADOLESCENTE               75                 0
6        ANCASH        ADULTO             3482                89
7        ANCASH  ADULTO MAYOR              933               238
8        ANCASH         JOVEN              915                 0
9        ANCASH          NIÑO              135                 0
10     APURIMAC   ADOLESCENTE               10                 0
11     APURIMAC        ADULTO              195                 2
12     APURIMAC  ADULTO MAYOR               34                 5

In [16]:
group = demographic.groupby(by=['COD', 'DEPARTAMENTO', 'SEXO']).sum()
group.reset_index(inplace=True)

names = group['COD'].unique()
depa = group['DEPARTAMENTO'].unique()
sexs = group['SEXO'].unique()
values = []
for sex in sexs:
    vals = []
    for name in names:
        vals.append(group[((group['COD'] == name)&(group['SEXO'] == sex))]['TOTAL_POSITIVOS'].values[0])
    values.append(vals)

bars = []    
for sex, value in zip(sexs, values):
    bars.append(go.Bar(name=sex, x=depa, y=value))

fig = go.Figure(data=bars)
# Change the bar mode
fig.update_layout(title='Gráfico de Positivos por Sexo', barmode='stack', xaxis_tickangle=-45)
fig.show()    